In [14]:
import os 
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))
import pytorch_lightning as pl
import torch
from model.model_interface import LLM
from dataset.supervised_dataset import SupervisedDataset
import torch.utils.data as tud
from lightning.pytorch.loggers import TensorBoardLogger
import torch.nn.functional as F
from utils.gpthook import TraceDict
import os 
import random
from tqdm.notebook import tqdm

In [15]:
import json

alpaca_data = json.load(open("data/alpaca_data.json"))
batch_size = 4
gradient_accumulation_steps = 8
max_epochs = 3
warm_ratio = 0.03
warmup_t0 = int((len(alpaca_data) // (batch_size * gradient_accumulation_steps)+1) * max_epochs * warm_ratio) + 1
print(warmup_t0)

5


In [40]:
# LLM Config
llm_config = {
    "model_name": "gpt2-xl",
    "optimizer": "adam_w",
    "lr": 2e-5,
    "lr_lambda" : "cosine",
    "warmup_t0" : warmup_t0,
    "weight_decay": 0.0,
}

In [41]:
mt = LLM(**llm_config)

In [30]:
# Dataset config
dl_config = {
    "batch_size": 64,
    "num_workers": 1
}
data_path= "data/alpaca_data.json"
size = 100

In [19]:
dst = SupervisedDataset(f"{data_path}", mt.tokenizer)

In [20]:
from transformers import AutoTokenizer
import torch.utils.data as tud
from dataset.supervised_dataset import SupervisedDataset
tokenizer = AutoTokenizer.from_pretrained("decapoda-research/llama-7b-hf")
tokenizer.add_special_tokens(
    {
        "eos_token": "</s>",
        "bos_token": "</s>",
        "unk_token": "</s>",
    }
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'
dst = SupervisedDataset("/nvme/yangyuchen1/coding/gpt_re/data/alpaca_data_sample.json", tokenizer)
train_dl = tud.DataLoader(dst, shuffle=True, collate_fn=dst.collate_fn, batch_size=8)
for d in train_dl:
    print(d[0].shape,d[2].shape)
    break
vocab = tokenizer.get_vocab()
print(len(vocab))

torch.Size([8, 158]) torch.Size([8, 158])
32000


In [31]:
train_dl = tud.DataLoader(dst, shuffle=True, collate_fn=dst.collate_fn, **dl_config)

In [22]:
# default logger used by trainer (if tensorboard is installed)
logger = TensorBoardLogger(save_dir=os.getcwd(), version=1, name="lightning_logs")

In [42]:
# Trainer config
trainer_config = {
    "precision" : "16-mixed",
    "accelerator" : "gpu",
    "devices" : [1,2,3,4],
    "strategy" : "fsdp",
    "max_epochs": 3,
    "accumulate_grad_batches" : 8
}

torch.set_float32_matmul_precision('medium')
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

In [43]:
from pytorch_lightning.callbacks import LearningRateMonitor
lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = pl.Trainer(**trainer_config, logger=logger, callbacks=[lr_monitor])

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(mt, train_dl)